# Secure Data Access To S3 Bucket via IAM Policy using VPC

In [1]:
import boto3
import sagemaker

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3 = boto3.Session().client(service_name='s3', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
role_name = role.split('/')[-1]
print(role_name)

AmazonSageMaker-ExecutionRole-20191006T135881


In [3]:
import time
timestamp = int(time.time())

In [4]:
bucket_secure_name = 'bucket-secure-iam-policy-vpc-{}'.format(timestamp)
print(bucket_secure_name)

bucket-secure-iam-policy-vpc-1608485380


In [5]:
!aws s3 mb s3://$bucket_secure_name

make_bucket: bucket-secure-iam-policy-vpc-1608485380


In [6]:
!aws s3 ls s3://$bucket_secure_name

In [7]:
policy_name='DSOAWS_Secure_IAMPolicy_VPC_{}'.format(timestamp)
print(policy_name)

DSOAWS_SecureBucket_Policy_VPC_1608485380


In [9]:
different_vpc_id='blah'

In [10]:
# Create the IAM policy
iam_policy_deny = {
        'Version': '2012-10-17',
        'Statement': [{
            'Sid': '',
            'Effect': 'Deny',
            'Action': [
                's3:ListBucket'
            ],
            'Resource': [
                'arn:aws:s3:::{}'.format(bucket_secure_name)
            ],
            'Condition': {
                'StringNotEquals': {
                    'aws:sourceVpc': different_vpc_id
                }
            }
            
        }]
    }

In [11]:
import json 

response = iam.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(iam_policy_deny)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '26199d93-8434-4b91-a1b0-c96645b4ec90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '26199d93-8434-4b91-a1b0-c96645b4ec90', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sun, 20 Dec 2020 17:29:58 GMT'}, 'RetryAttempts': 0}}


In [12]:
!aws s3 ls s3://$bucket_secure_name


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


# Remove the Policy

In [13]:
response = iam.delete_role_policy(
    RoleName=role_name,
    PolicyName=policy_name
)
print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': 'af10f758-5afc-4b4b-9f7b-6bc48a0b8570', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'af10f758-5afc-4b4b-9f7b-6bc48a0b8570', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 20 Dec 2020 17:31:50 GMT'}, 'RetryAttempts': 0}}


# Verify Access After Removing the Policy

In [14]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [15]:
!aws s3 rm s3://$bucket_secure_name